Step 1: download and convert dataset from CSV to Parquet

In [7]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [77]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask.distributed import Client, LocalCluster
import dask

In [ ]:
import time

In [78]:
print('pandas version: %s' % pd.__version__)
print('numpy version: %s' % np.__version__)
print('dask version: %s' % dask.__version__)

pandas version: 1.3.4
numpy version: 1.20.3
dask version: 2021.10.0


https://docs.databricks.com/_static/notebooks/koalas-benchmark-distributed-execution.html?_ga=2.216403934.95291449.1648935555-599276868.1645477063

In [27]:
filename = "taxi_dataset.txt"

In [122]:
with open(filename) as file:
    csv_files = [line.rstrip() for line in file]
# only choose yellow taxis
yellow = list(filter(lambda x: "yellow" in x, csv_files))

In [124]:
# make the list small for now
yellow = ['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [125]:
yellow

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [ ]:
import re
pattern = '/[^\/]*\.csv$/gm'

for csv_url in yellow:
#     get the filename only so I can sort by month / year
    csv_name = re.findall(r"[^\/]*\.csv$",csv_url)
    df = dd.read_csv(csv_url,dtype={'tolls_amount': 'float64'})

#     df = dd.read_csv(csv_url,
#                 dtype={'RatecodeID': 'float64',
#        'VendorID': 'float64',
#        'passenger_count': 'float64',
#        'payment_type': 'float64',
#        'tolls_amount': 'float64',
#        'trip_type': 'float64'
#       })
    df = df.repartition(npartitions=4)
    df.to_parquet(f'./tmp/trip_data_{csv_name}', write_index=False)


/Users/mk/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/Users/mk/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(


Read in parquet to dask

In [79]:
ddf = dd.read_parquet(
    "./tmp/", 
    storage_options={"anon": True, 'use_ssl': True}
)


In [80]:
ddf.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,0.5,0.5,0.00,0.0,NaN,0.3,6.80,2.0,1.0,0.00
1,2.0,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,0.5,0.5,2.81,0.0,NaN,0.3,16.86,1.0,1.0,2.75
2,2.0,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,0.5,0.5,1.00,0.0,NaN,0.3,8.30,1.0,1.0,0.00
3,2.0,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,0.5,0.5,0.00,0.0,NaN,0.3,9.30,2.0,1.0,0.00
4,2.0,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,0.0,-0.5,0.00,0.0,NaN,-0.3,-52.80,3.0,1.0,0.00


In [81]:
# try to do things

In [82]:
ddf = ddf[(ddf.tip_amount >= 1) & (ddf.tip_amount <= 5)]

In [83]:
ddf.persist()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
npartitions=4,,,,,,,,,,,,,,,,,,,,
,float64,object,object,object,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# benchmarks

In [105]:
%%time
# read_file_parquet
df = dd.read_parquet(
    "./tmp/", 
    storage_options={"anon": True, 'use_ssl': True}
)

CPU times: user 9.57 ms, sys: 2.19 ms, total: 11.8 ms
Wall time: 10.1 ms


In [106]:
%%time
# count
len(df)

CPU times: user 19 ms, sys: 13.7 ms, total: 32.7 ms
Wall time: 25.5 ms


76518

In [107]:
%%time
# count_index_length
len(df.index)

CPU times: user 99.3 ms, sys: 39.3 ms, total: 139 ms
Wall time: 104 ms


76518

In [108]:
%%time
# mean
df.fare_amount.mean().compute()

CPU times: user 24.7 ms, sys: 6.83 ms, total: 31.5 ms
Wall time: 25.3 ms


19.939777176612036

In [109]:
%%time
# standard_deviation
df.fare_amount.std().compute()

CPU times: user 30.7 ms, sys: 9.6 ms, total: 40.3 ms
Wall time: 31.9 ms


14.98424075460877

In [110]:
%%time
# mean_of_sum
(df.fare_amount + df.tip_amount).mean().compute()

CPU times: user 29.7 ms, sys: 6.57 ms, total: 36.3 ms
Wall time: 29.7 ms


21.44627656237748

In [111]:
%%time
# sum_columns
(df.fare_amount + df.tip_amount).compute()

CPU times: user 23.1 ms, sys: 7.47 ms, total: 30.6 ms
Wall time: 23.4 ms


0         5.50
1        12.81
2         7.00
3         8.00
4       -52.00
         ...  
19125    17.01
19126    30.02
19127    15.64
19128    53.42
19129    24.12
Length: 76518, dtype: float64

In [117]:
%%time
# mean_of_product
(df.fare_amount * df.tip_amount).mean().compute()

CPU times: user 25.1 ms, sys: 6.75 ms, total: 31.9 ms
Wall time: 24.2 ms


34.512497963877784

In [118]:
%%time
# product_columns
(df.fare_amount * df.tip_amount).compute()

CPU times: user 23.1 ms, sys: 6.72 ms, total: 29.8 ms
Wall time: 25.3 ms


0          0.0000
1         28.1000
2          6.0000
3          0.0000
4         -0.0000
           ...   
19125      0.0000
19126     74.9925
19127     35.4475
19128    139.3425
19129     58.7675
Length: 76518, dtype: float64

In [119]:
%%time
# value_counts
df.fare_amount.value_counts().compute()

CPU times: user 30.8 ms, sys: 9.41 ms, total: 40.2 ms
Wall time: 35.1 ms


7.00      2252
6.50      1942
6.00      1898
8.00      1809
7.50      1709
          ... 
27.73        1
27.64        1
27.56        1
27.53        1
284.00       1
Name: fare_amount, Length: 3399, dtype: int64

In [120]:
%%time
# mean_of_complicated_arithmetic_operation
theta_1 = df.start_lon
phi_1 = df.start_lat
theta_2 = df.end_lon
phi_2 = df.end_lat
temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
       + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
ret.mean().compute()

AttributeError: 'DataFrame' object has no attribute 'start_lon'

In [116]:
%%time
# complicated_arithmetic_operation


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


In [ ]:
%%time
# groupby_statistics


In [ ]:
%%time
# join_count


In [ ]:
%%time
# join_data


In [84]:
# "operations" as defined by databricks.

In [85]:
# # complex arithmetic
# np.sin ... np.cos ... np.arctan2

In [86]:
len(ddf)

37878

In [87]:
len(ddf.index)

37878

In [ ]:
df.groupby(by='series_c').agg(... ['mean', 'std'] ...)

In [ ]:
# from databricks documentation.

In [ ]:
def operations(df):
    # complex arithmetic
    np.sin ... np.cos ... np.arctan2
    # count
    len(df)
    # count index
    len(df.index)
    # groupby statistics
    df.groupby(by='series_c').agg(... ['mean', 'std'] ...)
    # join
    merge(df1, df2)
    # join count
    len(merge(df1, df2))
    # mean
    df.series_a.mean()
    # mean of complex arithmetic
    (np.sin ... np.cos ... np.arctan2).mean()
    # mean of series addition
    (df.series_a + df.series_b).mean()
    # mean of series multiplication
    (df.series_a * df.series_b).mean()
    # read file
    read_parquet(...)
    # series addition
    df.series_a + df.series_b
    # series multiplication
    df.series_a * df.series_b
    # standard derivation
    df.series_a.std()
    # value counts
    df.series_a.value_counts()

In [ ]:
# Dask
df = df[(df.tip_amt >= 1) & (df.tip_amt <= 5)]
df = dask_client.persist(df)
wait(df) # Make sure data is cached.
operations(df)
